In [ ]:
from TradingviewData import TredingViewData,Interval
import time
import os
import warnings
import datetime
import numpy as np
# from tradingview_ta import TA_Handler, Interval, Exchange

warnings.filterwarnings("ignore")

try:
    import xlwings as xw
except (ModuleNotFoundError, ImportError):
    print("xlwings module not found")
    os.system(f"{sys.executable} -m pip install -U xlwings")
finally:
    import xlwings as xw
    
try:
    import requests
except (ModuleNotFoundError, ImportError):
    print("requests module not found")
    os.system(f"{sys.executable} -m pip install -U requests")
finally:
    import requests

try:
    import pandas as pd
except (ModuleNotFoundError, ImportError):
    print("pandas module not found")
    os.system(f"{sys.executable} -m pip install -U pandas")
finally:
    import pandas as pd
    
try:
    from bs4 import BeautifulSoup
except (ModuleNotFoundError, ImportError):
    print("BeautifulSoup module not found")
    os.system(f"{sys.executable} -m pip install -U beautifulsoup4")
finally:
    from bs4 import BeautifulSoup
    
    
def calculate_rsi(prices, period=14):
    deltas = np.diff(prices)
    seed = deltas[:period+1]
    up = seed[seed>=0].sum()/period
    down = -seed[seed<0].sum()/period
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:period] = 100. - 100./(1.+rs)
    for i in range(period, len(prices)):
        delta = deltas[i-1]
        if delta>0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta
        up = (up*(period-1) + upval)/period
        down = (down*(period-1) + downval)/period
        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)
    return rsi[-1]

def calculate_bb_bands(prices, window, num_std):
    rolling_mean = np.convolve(prices, np.ones(window)/window, mode='valid')
    rolling_std = np.std(prices[-window:])
    upper_band = rolling_mean + (num_std * rolling_std)
    lower_band = rolling_mean - (num_std * rolling_std)
    return upper_band.tolist(), lower_band.tolist()


def calculate_volume_sma(stock_data, window):
    volume_sma = stock_data['volume'].rolling(window=window).mean()
    return volume_sma


Charting_Link = "https://chartink.com/screener/"
Charting_url = 'https://chartink.com/screener/process'


cond_bullish =  "( {cash} ( ( {33489} ( latest rsi( 14 ) >= 60 and weekly rsi( 14 ) >= 60 and monthly rsi( 14 ) >= 55 and latest close >= 100 and latest close <= 5000 ) ) ) ) "

cond_bearish = "( {cash} ( ( {33489} ( latest rsi( 14 ) <= 40 and weekly rsi( 14 ) <= 40 and monthly rsi( 14 ) <= 45 and latest close >= 100 and latest close <= 5000 ) ) ) ) "

def GetDataFromChartink(payload):
    payload = {'scan_clause': payload}
    
    with requests.Session() as s:
        r = s.get(Charting_Link)
        soup = BeautifulSoup(r.text, "html.parser")
        csrf = soup.select_one("[name='csrf-token']")['content']
        s.headers['x-csrf-token'] = csrf
        r = s.post(Charting_url, data=payload)

        df = pd.DataFrame()
        for item in r.json()['data']:
            df = df.append(item, ignore_index=True)
    return df


try:
    if not os.path.exists('Chartink_Result.xlsm'):
        wb = xw.Book()
        wb.save('Chartink_Result.xlsm')
        wb.close()

    wb = xw.Book('Chartink_Result.xlsm')
    try:
        result = wb.sheets('Chartink_Result')
    except Exception as e:
        wb.sheets.add('Chartink_Result')
        result = wb.sheets('Chartink_Result')
except Exception as e:
    pass




data_bearish = GetDataFromChartink(cond_bearish)

data_bullish = GetDataFromChartink(cond_bullish)
    # Reset the Indexes Output
data_bullish.reset_index()
data_bearish.reset_index()
    # request TradingviewData
request = TredingViewData()


data = pd.concat([data_bearish,data_bullish]  ,ignore_index=True)
    
data = data.drop(['bsecode','sr','volume'], axis=1)

data['accure_time'] = str("")

    
while True:

    
    for i , row in data_bullish.iterrows():
   
        try:
            # Download the Stocks Data From the Tradingview 
            five_data_bullish = request.get_hist(symbol=row['nsecode'],exchange='NSE',interval=Interval.min_5,n_bars=78)
            fifteen_data_bullish = request.get_hist(symbol=row['nsecode'],exchange='NSE',interval=Interval.min_15,n_bars=50)
            thirty_data_bullish = request.get_hist(symbol=row['nsecode'],exchange='NSE',interval=Interval.min_30,n_bars=50)
        
        except:
            print("Error occurred at stock Data for " + row['nsecode'] , end="\r")
            
        
        try:
                
                five_close = five_data_bullish['close'][-1]
                five_open = five_data_bullish['open'][-1]
                thirty_high = thirty_data_bullish['close'][-1]

                # calculate the 5 Min and 30 Min RSI Values
                rsi_5 = calculate_rsi(five_data_bullish['close'])
                rsi_30 = calculate_rsi(thirty_data_bullish['close'])

                # calculate the 5 Min and 15 Min Boolinger Band Values
                upper_band_5 , lower_band_5 = calculate_bb_bands(five_data_bullish['close'],20,1.5)
                upper_band_15 , lower_band_15 = calculate_bb_bands(fifteen_data_bullish['close'],20,1.5)


                # calculate the 20 sma volume value
                volume = five_data_bullish['volume'][-1]
                sma_volume = calculate_volume_sma(five_data_bullish,20)
                
                
                # add some data into the main table
                data.at[i,'time'] = datetime.datetime.now().strftime('%H:%M:%S')
                data.at[i,'close'] = five_close
                data.at[i,'rsi'] = rsi_5
               
                print(i , row['nsecode'] , end = "\r")
                
                
                if (rsi_5 > 57 and rsi_30 > 55 and five_close >= thirty_high and volume > sma_volume and five_close > upper_band_15[-1] and five_open < upper_band_5[-1] and five_close > upper_band_5[-1]):
                    data.at[i,'Status'] = "True"
                    data.at[i,'accure_time'] += "|" + str(datetime.datetime.now().strftime('%H:%M:%S'))
                else:
                    data.at[i,'Status'] = "False"
                
        except:
             print("Error occurred at stock Calculation for " + row['nsecode'] , end="\r")
                
                
        
                
    
    
    for i , row in data_bearish.iterrows():
   
        try:
            # Download the Stocks Data From the Tradingview 
            five_data_bearish = request.get_hist(symbol=row['nsecode'],exchange='NSE',interval=Interval.min_5,n_bars=78)
            fifteen_data_bearish = request.get_hist(symbol=row['nsecode'],exchange='NSE',interval=Interval.min_15,n_bars=50)
            thirty_data_bearish = request.get_hist(symbol=row['nsecode'],exchange='NSE',interval=Interval.min_30,n_bars=50)
            
        except:
            print("Error occurred at stock Data for " + row['nsecode'] , end="\r")
            
        
        try:
                five_close = five_data_bearish['close'][-1]
                five_open = five_data_bearish['open'][-1]
                thirty_low = thirty_data_bearish['close'][-1]

                # calculate the 5 Min and 30 Min RSI Values
                rsi_5 = calculate_rsi(five_data_bearish['close'])
                rsi_30 = calculate_rsi(thirty_data_bearish['close'])


                # calculate the 5 Min and 15 Min Boolinger Band Values
                upper_band_5 , lower_band_5 = calculate_bb_bands(five_data_bearish['close'],20,1.5)
                upper_band_15 , lower_band_15 = calculate_bb_bands(fifteen_data_bearish['close'],20,1.5)


                # calculate the 20 sma volume value
                volume = five_data_bearish['volume'][-1]
                sma_volume = calculate_volume_sma(five_data_bearish,20)
                
#                 # get technical analysis
#                 tech = TA_Handler(symbol=row['nsecode'],screener="india",exchange="NSE",interval=Interval.INTERVAL_5_MINUTES)
#                 data.at[i,'technical'] = tech.get_analysis().summary['RECOMMENDATION']
                
                
                # add some data into the main table
                data.at[i,'time'] = datetime.datetime.now().strftime('%H:%M:%S')
                data.at[i,'close'] = five_close
                data.at[i,'rsi'] = rsi_5
                print(i , row['nsecode'] , end = "\r")
                
                if (rsi_5 < 43 and rsi_30 < 45 and five_close <= thirty_low and volume > sma_volume and five_close < lower_band_15[-1] and five_open > lower_band_5[-1] and five_close < lower_band_5[-1]):
                    data.at[i,'Status'] = "True"
                    data.at[i,'accure_time'] += "|" + str(datetime.datetime.now().strftime('%H:%M:%S'))
                else:
                    data.at[i,'Status'] = "False"
                
                
        except:
             print("Error occurred at stock Calculation for " + row['nsecode'] , end="\r")
                

                

    data = data.sort_values(by='per_chg',ascending=False)
    
#     print(f"\n\n{data}")
    
    try:
        result.range('a:h').value = None
        result.range('a1').options(index=False).value = data
    except Exception as e:
        pass